In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import numpy as np
import pickle
import os

class FileData(object):
    def __init__(self, path):
        self.path = path
        with open(path, encoding='utf-16') as f:
          self.data = f.read()
          #print(self.data)

ABSOLUTE_PATH = r"/content/drive/MyDrive/vnese-spell-correction-lstm/Train_Full"

c_tri =  "/Chinh tri Xa hoi"

doi_song = "/Doi song"

khoa_hoc = "/Khoa hoc"

kinh_doanh = "/Kinh doanh"

p_luat = "/Phap luat"

suc_khoe = "/Suc khoe"

the_gioi = "/The gioi"

the_thao = "/The thao"

van_hoa = "/Van hoa"

vi_tinh = "/Vi tinh"
corpus = [c_tri, doi_song, khoa_hoc, kinh_doanh, p_luat, suc_khoe, the_gioi, the_thao, van_hoa, vi_tinh]

#Extract folder path
for folder_path in range(len(corpus)):
    corpus[folder_path] = ABSOLUTE_PATH + corpus[folder_path]

file_list = []

#Extracting text from corpus
for folder_path in corpus:
    count = 0
    for name in os.listdir(folder_path):
        count +=1
        if count == 1500:
          break
        path = os.path.join(folder_path, name)
        if not os.path.isfile(path):
            continue
        file = FileData(path)
        file_list.append( file.data )

print('Corpus length: ', len(file_list))

#Save extracted corpus as Pickle file
path_corpus = r"/content/drive/MyDrive/vnese-spell-correction-lstm/train_corpus.pkl"

with open(path_corpus, 'wb') as pickle_file:
    pickle.dump(file_list, pickle_file)

In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
import re
import numpy as np
import pickle
from unidecode import unidecode
import itertools
from nltk import ngrams
from tqdm import tqdm

path_corpus = r"/content/drive/MyDrive/vnese-spell-correction-lstm/train_corpus.pkl"

with open(path_corpus, "rb") as f:
    data = pickle.load(f)

alphabet = '^[ _abcdefghijklmnopqrstuvwxyz0123456789áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ!\"\',\-\.:;?_\(\)]+$'

#Extracting sentence from corpus
def latin_extract(data):

    # extract Latin- characters only
    latin_extract_data=[]
    # duyet qua tung van ban
    for i in data:
      if i == 1:
        break
      # thay the xuong dong la dau cham ket thuc
      i=i.replace("\n",".")
      # tach van ban theo dau cham ket thuc
      sentences=i.split(".")
      for j in sentences:
          if len(j.split()) > 2 and re.match(alphabet, j.lower()):

              latin_extract_data.append(j)

    return latin_extract_data

training_data = latin_extract(data)
i = 100
#Listing all typos, regional dialects
letters=list("abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
letters2=list("abcdefghijklmnopqrstuvwxyz")

typo={"ă":"aw","â":"aa","á":"as","à":"af","ả":"ar","ã":"ax","ạ":"aj","ắ":"aws","ổ":"oor","ỗ":"oox","ộ":"ooj","ơ":"ow",
"ằ":"awf","ẳ":"awr","ẵ":"awx","ặ":"awj","ó":"os","ò":"of","ỏ":"or","õ":"ox","ọ":"oj","ô":"oo","ố":"oos","ồ":"oof",
"ớ":"ows","ờ":"owf","ở":"owr","ỡ":"owx","ợ":"owj","é":"es","è":"ef","ẻ":"er","ẽ":"ex","ẹ":"ej","ê":"ee","ế":"ees","ề":"eef",
"ể":"eer","ễ":"eex","ệ":"eej","ú":"us","ù":"uf","ủ":"ur","ũ":"ux","ụ":"uj","ư":"uw","ứ":"uws","ừ":"uwf","ử":"uwr","ữ":"uwx",
"ự":"uwj","í":"is","ì":"if","ỉ":"ir","ị":"ij","ĩ":"ix","ý":"ys","ỳ":"yf","ỷ":"yr","ỵ":"yj","đ":"dd",
"Ă":"Aw","Â":"Aa","Á":"As","À":"Af","Ả":"Ar","Ã":"Ax","Ạ":"Aj","Ắ":"Aws","Ổ":"Oor","Ỗ":"Oox","Ộ":"Ooj","Ơ":"Ow",
"Ằ":"AWF","Ẳ":"Awr","Ẵ":"Awx","Ặ":"Awj","Ó":"Os","Ò":"Of","Ỏ":"Or","Õ":"Ox","Ọ":"Oj","Ô":"Oo","Ố":"Oos","Ồ":"Oof",
"Ớ":"Ows","Ờ":"Owf","Ở":"Owr","Ỡ":"Owx","Ợ":"Owj","É":"Es","È":"Ef","Ẻ":"Er","Ẽ":"Ex","Ẹ":"Ej","Ê":"Ee","Ế":"Ees","Ề":"Eef",
"Ể":"Eer","Ễ":"Eex","Ệ":"Eej","Ú":"Us","Ù":"Uf","Ủ":"Ur","Ũ":"Ux","Ụ":"Uj","Ư":"Uw","Ứ":"Uws","Ừ":"Uwf","Ử":"Uwr","Ữ":"Uwx",
"Ự":"Uwj","Í":"Is","Ì":"If","Ỉ":"Ir","Ị":"Ij","Ĩ":"Ix","Ý":"Ys","Ỳ":"Yf","Ỷ":"Yr","Ỵ":"Yj","Đ":"Dd"}

# dia phuong
region={"ẻ":"ẽ","ẽ":"ẻ","ũ":"ủ","ủ":"ũ","ã":"ả","ả":"ã","ỏ":"õ","õ":"ỏ","i":"j"}
region2={"s":"x","l":"n","n":"l","x":"s","d":"gi","S":"X","L":"N","N":"L","X":"S","Gi":"D","D":"Gi"}

# nguyen am
vowel=list("aeiouyáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵ")

# viet tat
acronym={"không":"ko"," anh":" a","em":"e","biết":"bít","giờ":"h","gì":"j","muốn":"mún","học":"hok","yêu":"iu",
         "chồng":"ck","vợ":"vk"," ông":" ô","được":"đc","tôi":"t",
         "Không":"Ko"," Anh":" A","Em":"E","Biết":"Bít","Giờ":"H","Gì":"J","Muốn":"Mún","Học":"Hok","Yêu":"Iu",
         "Chồng":"Ck","Vợ":"Vk"," Ông":" Ô","Được":"Đc","Tôi":"T",}

# teencode
teen={"ch":"ck","ph":"f","th":"tk","nh":"nk",
      "Ch":"Ck","Ph":"F","Th":"Tk","Nh":"Nk"}

# function for adding mistake( noise)
def teen_code(sentence, pivot):
    random = np.random.uniform(0,1,1)[0]
    new_sentence=str(sentence)
    if random>pivot:
        for word in acronym.keys():
            if re.search(word, new_sentence):
                random2 = np.random.uniform(0,1,1)[0]
                if random2 <0.5:
                    new_sentence=new_sentence.replace(word,acronym[word])
        for word in teen.keys():
            if re.search(word, new_sentence):
                random3 = np.random.uniform(0,1,1)[0]
                if random3 <0.05:
                    new_sentence=new_sentence.replace(word,teen[word])
        return new_sentence
    else:
        return sentence


def add_noise(sentence, pivot1,pivot2):
    sentence=teen_code(sentence,0.5)
    noisy_sentence = ""
    i = 0
    while i < len(sentence):
        if sentence[i] not in letters:
            noisy_sentence+=sentence[i]
        else:
            random = np.random.uniform(0,1,1)[0]
            if random < pivot1:
                noisy_sentence+=(sentence[i])
            elif random<pivot2:
                if sentence[i] in typo.keys() and sentence[i] in region.keys():
                    random2=np.random.uniform(0,1,1)[0]
                    if random2<=0.4:
                        noisy_sentence+=typo[sentence[i]]
                    elif random2<0.8:
                        noisy_sentence+=region[sentence[i]]
                    elif random2<0.95 :
                        noisy_sentence+=unidecode(sentence[i])
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in typo.keys():
                    random3=np.random.uniform(0,1,1)[0]
                    if random3<=0.6:
                        noisy_sentence+=typo[sentence[i]]
                    elif random3<0.9 :
                        noisy_sentence+=unidecode(sentence[i])
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in region.keys():
                    random4=np.random.uniform(0,1,1)[0]
                    if random4<=0.6:
                        noisy_sentence+=region[sentence[i]]
                    elif random4<0.85 :
                        noisy_sentence+=unidecode(sentence[i])
                    else:
                        noisy_sentence+=sentence[i]
                elif i<len(sentence)-1 :
                    if sentence[i] in region2.keys() and (i==0 or sentence[i-1] not in letters) and sentence[i+1] in vowel:
                        random5=np.random.uniform(0,1,1)[0]
                        if random5<=0.9:
                            noisy_sentence+=region2[sentence[i]]
                        else:
                            noisy_sentence+=sentence[i]
                    else:
                        noisy_sentence+=sentence[i]

            else:
                new_random = np.random.uniform(0,1,1)[0]
                if new_random <=0.33:
                    if i == (len(sentence) - 1):
                        continue
                    else:
                        noisy_sentence+=(sentence[i+1])
                        noisy_sentence+=(sentence[i])
                        i += 1
                elif new_random <= 0.66:
                    random_letter = np.random.choice(letters2, 1)[0]
                    noisy_sentence+=random_letter
                else:
                    pass

        i += 1
    return noisy_sentence

def extract_phrases(text):
    return re.findall(r'\w[\w ]+', text)

def _extract_phrases(data):
    phrases = itertools.chain.from_iterable(extract_phrases(text) for text in data)
    phrases = [p.strip() for p in phrases if len(p.split()) > 1]

    return phrases

phrases = _extract_phrases(training_data)

#Generate Bi-gram

#A Vietnamese word do not contain more than 7 characters, so an bi-gram do not have more than 15 characters
NGRAM = 2
MAXLEN = 40

def gen_ngrams(words, n=2):
    return ngrams(words.split(), n)

def generate_bi_grams(phrases):
    list_ngrams = []
    for p in tqdm(phrases):

      # neu khong nham trong bang chu cai thi bo qua
      if not re.match(alphabet, p.lower()):
        continue

      # tach p thanh cac bi gram
      for ngr in gen_ngrams(p, NGRAM):
        if len(" ".join(ngr)) < MAXLEN:
          list_ngrams.append(" ".join(ngr))

    return list_ngrams

list_ngrams = generate_bi_grams(phrases)

print(len(list_ngrams))

alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']

def encoder_data(text, maxlen=MAXLEN):
        #print("Maxlen", maxlen)
        text = "\x00" + text
        #print("text", text)
        x = np.zeros((maxlen, len(alphabet)))
        #print("X ban dau", x)
        for i, c in enumerate(text[:maxlen]):
            x[i, alphabet.index(c)] = 1
        if i < maxlen - 1:
          for j in range(i+1, maxlen):
            x[j, 0] = 1
        return x

def decoder_data(x):
    x = x.argmax(axis=-1)
    #print("x hien tai", x)
    dem = ''.join(alphabet[i] for i in x)
    #print("Do dai cau van", len(dem))

    return dem


100%|██████████| 666584/666584 [00:17<00:00, 37161.84it/s]

5793398


In [ ]:
print(data[0:4])
print(training_data[0:4])
print(phrases[0:5])

["Các nhà khoa học nói về ô nhiễm tại hố chôn gà dịch \nTrong những ngày qua, có nhiều ý kiến phản ánh của người dân về tình trạng ô nhiễm tại các điểm chôn gà chết của TP HCM. Ở các hố chôn, nước dịch trào ra, thoát khí nặng mùi, nhất là vào những hôm trời nóng bức. VnExpress đã trao đổi với một số nhà khoa học và quản lý môi trường về vấn đề này.\nGiáo sư, Tiến sĩ Tưởng Thị Hội, giảng viên Viện Khoa học và công nghệ Môi trường, ĐH Bách Khoa Hà Nội: Động vật khi chôn sâu trong lòng đất sẽ bị phân hủy dưới dạng hiếu khí và yếm khí, tạo ra những hợp chất dạng khác nhau như thể lỏng hoặc thể khí. Các dịch lỏng phát sinh có thể ngấm vào đất và gây ô nhiễm nguồn nước. Điều này trước kia cũng đã xảy ra ở nghĩa trang Văn Điển, Hà Nội. Các khí tạo thành trong quá trình phân hủy chất hữu cơ như CH4, NH3, H2S, CO2 thoát ra ngoài môi trường sẽ tạo ra mùi hôi, gây độc hại đến môi trường sống.\nGiáo sư Nguyễn Công Mẫn, Phó chủ tịch Viện Địa kỹ thuật thuộc Liên hiệp các Hội khoa học kỹ thuật Việt N

In [ ]:
from keras.preprocessing.text import Tokenizer

def build_vocab(sentences):
    tokenizer = Tokenizer(filters='')
    tokenizer.fit_on_texts(sentences)
    vocab = tokenizer.word_index
    return vocab

# Sử dụng hàm để xây dựng vocab từ danh sách các câu văn
vocab = build_vocab(phrases)

# In vocab
print("Vocab:")
print(len(vocab))



Vocab:
39029


In [ ]:
pip install tensorflow tensorflow-addons

In [ ]:
import torch
import torch.nn as nn
import math

class TransformerEncoderModel(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_encoder_layers, max_seq_length, dropout=0.1):
        super(TransformerEncoderModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout, max_seq_length)

        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead),
            num_layers=num_encoder_layers
        )

        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x = self.positional_encoding(x)

        output = self.transformer_encoder(x)
        output = output.mean(dim=0)  # Global average pooling

        return self.fc(output)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=512):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# Tham số của mô hình
vocab_size = len(alphabet)  # Kích thước từ vựng
d_model = 256  # Kích thước embedding
nhead = 8  # Số lượng attention heads trong mỗi layer
num_encoder_layers = 6  # Số lượng layer trong encoder
max_seq_length = MAXLEN  # Độ dài tối đa của chuỗi đầu vào

# Tạo và compile mô hình
transformer_model = TransformerEncoderModel(
    vocab_size=vocab_size,
    d_model=d_model,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    max_seq_length=max_seq_length
)

# In thông tin về mô hình
print(transformer_model)

TransformerEncoderModel(
  (embedding): Embedding(199, 256)
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=256, out_features=199, bias=True)
)


In [ ]:
from tensorflow.keras import layers

class TransformerBlock(layers.Layer):
  def __init__(self, embed_dim, numh_heads, ff_dim, rate=0.1):
    super().__init__
    self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    self.ffn = keras.Sequential(
        [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
    )
    self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
    self.dropout1 = layers.Dropout(rate)
    self.dropout2 = layers.Dropout(rate)

  def call(self, inputs, training):
    attn_output = self.att(inputs, inputs)
    attn_output = self.dropout1(attn_output, training = training)
    out1 = self.layernorm1(inputs + attn_output)
    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training = training)
    return self.layernorm2(out1, ffn_output)


In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
  def __init__(self, maxlen, vocab_size, embed_dim):
    super().__init__
    self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
    self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

  def call(self, x):
    maxlen = tf.shape(x)[-1]
    positions = tf.range(start=0, limit=maxlen, delta=1)
    positions = self.pos_emb(positions)
    x = self.token_emb(x)
    return x + positions


In [ ]:
from tensorflow import keras

sequence_len = 200
embed_dim = 128
num_heads = 6
ff_dim = 128

inputs = layers.Input(shape=(sequence_len,))
embedding_layer = TokenAndPositionEmbedding(sequence_len, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dims)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

sequence_len = 200
embed_dim = 128
num_heads = 6
ff_dim = 128
vocab_size = 10000  # Replace with the actual vocabulary size

inputs = layers.Input(shape=(sequence_len,))
embedding_layer = TokenAndPositionEmbedding(sequence_len, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = models.Model(inputs=inputs, outputs=outputs)
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

# Display the model summary
model.summary()

# Split training data
train_data, valid_data = train_test_split(list_ngrams, test_size=0.2, random_state=42)

# Define batch size
BATCH_SIZE = 512

# Define data generation method
def generate_data(data, batch_size):
    cur_index = 0
    while True:
        x, y = [], []
        for i in range(batch_size):
            y.append(encoder_data(data[cur_index]))
            x.append(encoder_data(add_noise(data[cur_index], 0.94, 0.985)))
            cur_index += 1
            if cur_index > len(data) - 1:
                cur_index = 0
        yield np.array(x), np.array(y)

# Create data generators for training and validation
train_generator = generate_data(train_data, batch_size=BATCH_SIZE)
validation_generator = generate_data(valid_data, batch_size=BATCH_SIZE)

# Train the model and save the best weights
checkpointer = ModelCheckpoint(filepath=os.path.join('/content/drive/MyDrive/vnese-spell-correction-lstm/spelling_ver1.h5'),
                               save_best_only=True, verbose=1)

# Fit the model
model.fit(train_generator, steps_per_epoch=len(train_data)//BATCH_SIZE, epochs=5,
          validation_data=validation_generator, validation_steps=len(valid_data)//BATCH_SIZE,
          callbacks=[checkpointer])


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 token_and_position_embeddi  (None, 200, 128)          1305600   
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  (None, 200, 128)          429184    
 rmerBlock)                                                      
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_2 (Dropout)         (None, 128)               0     

In [ ]:
import tensorflow as tf
import os
from tensorflow.keras import layers
from tensorflow.keras import models
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, TimeDistributed, Dense,LSTM, Bidirectional
from keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Sửa lỗi trong layer SpellCorrectionTransformer
class SpellCorrectionTransformer(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(SpellCorrectionTransformer, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

        # Sửa đổi key_dim để phản ánh kích thước của đầu vào
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)


    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

# Assume you have vocab_size and sequence_len defined
vocab_size = 10000
sequence_len = 200

# Create the model
embed_dim = 128
num_heads = 6
ff_dim = 128

inputs = layers.Input(shape=(sequence_len,))
embedding_layer = TokenAndPositionEmbedding(sequence_len, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = SpellCorrectionTransformer(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

# Split training data
train_data, valid_data = train_test_split(list_ngrams, test_size=0.2, random_state=42)

# Define batch size
BATCH_SIZE = 512

# Define data generation method
def generate_data(data, batch_size):
    cur_index = 0
    while True:
        x, y = [], []
        for i in range(batch_size):
            y.append(encoder_data(data[cur_index]))
            x.append(encoder_data(add_noise(data[cur_index], 0.94, 0.985)))
            cur_index += 1
            if cur_index > len(data) - 1:
                cur_index = 0
        yield np.array(x), np.array(y)

# Create data generators for training and validation
train_generator = generate_data(train_data, batch_size=BATCH_SIZE)
validation_generator = generate_data(valid_data, batch_size=BATCH_SIZE)

# Train the model and save the best weights
checkpointer = ModelCheckpoint(filepath=os.path.join('/content/drive/MyDrive/vnese-spell-correction-lstm/spelling_transformer.h5'),
                               save_best_only=True, verbose=1)


model.fit(train_data, steps_per_epoch=len(train_data)//BATCH_SIZE, epochs=5,
          validation_data=valid_data, validation_steps=len(valid_data)//BATCH_SIZE,
          callbacks=[checkpointer])


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 200)]             0         
                                                                 
 token_and_position_embeddi  (None, 200, 128)          1305600   
 ng_3 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 spell_correction_transform  (None, 200, 128)          98554     
 er_3 (SpellCorrectionTrans                                      
 former)                                                         
                                                                 
 global_average_pooling1d_3  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                           

In [ ]:
from collections import Counter
from keras.models import load_model
from nltk import ngrams,word_tokenize
import numpy as np
import re
import string

model = load_model('/content/drive/MyDrive/vnese-spell-correction-lstm/spelling_ver1.h5')

NGRAM=2
MAXLEN=40
alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']
letters=list("abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
accepted_char=list((string.digits + ''.join(letters)))

def call(sentence):
    def encoder_data(text, maxlen=MAXLEN):
            text = "\x00" + text
            x = np.zeros((maxlen, len(alphabet)))
            for i, c in enumerate(text[:maxlen]):
                x[i, alphabet.index(c)] = 1
            if i < maxlen - 1:
              for j in range(i+1, maxlen):
                x[j, 0] = 1
            return x

    def decoder_data(x):
        x = x.argmax(axis=-1)
        return ''.join(alphabet[i] for i in x)

    def nltk_ngrams(words, n=2):
        return ngrams(words.split(), n)

    def guess(ngram):
        text = ' '.join(ngram)
        preds = model.predict(np.array([encoder_data(text)]), verbose=0)
        return decoder_data(preds[0]).strip('\x00')

    def correct(sentence):
        for i in sentence:
            if i not in accepted_char:
                sentence=sentence.replace(i," ")
        ngrams = list(nltk_ngrams(sentence, n=NGRAM))
        guessed_ngrams = list(guess(ngram) for ngram in ngrams)

        print("N gram", ngrams)
        print("guess", guessed_ngrams)

        candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
        for nid, ngram in (enumerate(guessed_ngrams)):
            for wid, word in (enumerate(re.split(' +', ngram))):
                candidates[nid + wid].update([word])

        output = ' '.join(c.most_common(1)[0][0] for c in candidates)
        return output

    guess = correct(sentence)

    return guess

# sentence = "Tôi là sinh vien trường đại họ cPhenikaa taji Hà Dông"
# sentence = 'Tôi phông iu thích hok ngôn ngữ mới'
sentence = 'tôi teen laf Tô Văn Tus, tooi ddang hojc lớp 5'

guess = call(sentence)
print(guess)

N gram [('tôi', 'teen'), ('teen', 'laf'), ('laf', 'Tô'), ('Tô', 'Văn'), ('Văn', 'Tus'), ('Tus', 'tooi'), ('tooi', 'ddang'), ('ddang', 'hojc'), ('hojc', 'lớp'), ('lớp', '5')]
guess ['tôi tên', 'tên là', 'là Tô', 'Tô Văn', 'Văn Tú', 'Tú ttôi', 'tôi đang', 'đang học', 'học lớp', 'lớp 5']
tôi tên là Tô Văn Tú ttôi đang học lớp 5
